In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL for NCAA stats
base_url = "https://www.ncaa.com"
stats_page = f"{base_url}/stats/baseball/d1"

# Function to get page content
def get_soup(url):
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    response.raise_for_status()  # Ensure request was successful
    return BeautifulSoup(response.text, "html.parser")

# Get main page content
soup = get_soup(stats_page)

# Find the dropdown container and extract stat URLs
dropdown = soup.find("select", {"id": "select-container-team"})
options = dropdown.find_all("option")

# Extract stat names and links
stat_links = {
    option.text.strip(): base_url + option["value"]
    for option in options if option.get("value")
}

In [2]:
def get_stat_dataframe(stat_name):
    """Fetches the specified stat table from multiple pages and returns a combined DataFrame,
    keeps 'Team' as string, and converts all other columns to float."""
    
    if stat_name not in stat_links:
        print(f"Stat '{stat_name}' not found. Available stats: {list(stat_links.keys())}")
        return None
    
    # Initialize the DataFrame to store all pages' data
    all_data = []
    page_num = 1  # Start from the first page

    while True:
        url = stat_links[stat_name]
        if page_num > 1:
            # Modify the URL to include the page number
            url = f"{url}/p{page_num}"
        
        # print(f"Fetching data for: {stat_name} (Page {page_num} - {url})")

        try:
            # Get stats page content
            soup = get_soup(url)

            # Locate table
            table = soup.find("table")
            if not table:
                print(f"No table found for {stat_name} on page {page_num}")
                break  # Exit the loop if no table is found (end of valid pages)

            # Extract table headers
            headers = [th.text.strip() for th in table.find_all("th")]

            # Extract table rows
            data = []
            for row in table.find_all("tr")[1:]:  # Skip header row
                cols = row.find_all("td")
                data.append([col.text.strip() for col in cols])

            all_data.extend(data)  # Add the data from this page to the list of all data
        
        except requests.exceptions.HTTPError as e:
            print(f"{stat_name} Done")
            break  # Exit the loop on HTTPError (page doesn't exist)
        except Exception as e:
            print(f"An error occurred: {e}")
            break  # Exit the loop on any other error

        page_num += 1  # Go to the next page

    # Convert to DataFrame
    if all_data:
        df = pd.DataFrame(all_data, columns=headers)

        # Convert all columns to float except "Team"
        for col in df.columns:
            if col != "Team":
                df[col] = pd.to_numeric(df[col], errors="coerce")  # Converts to float, invalid values become NaN

        return df
    else:
        print("No data collected.")
        return None

# Example usage
stat_name_input = "Batting Average"  # Change this to the desired stat
ba = get_stat_dataframe(stat_name_input)
ba["HPG"] = ba["H"] / ba["G"]
ba["ABPG"] = ba["AB"] / ba["G"]
ba["HPAB"] = ba["H"] / ba["AB"]
ba = ba.drop(columns=['Rank'])

stat_name_input = "Base on Balls"
bb = get_stat_dataframe(stat_name_input)
bb["BBPG"] = bb["BB"] / bb["G"]
bb = bb.drop(columns=['Rank', 'G'])

stat_name_input = "Double Plays Per Game"
dp = get_stat_dataframe(stat_name_input)
dp.rename(columns={"PG": "DPPG"}, inplace=True)
dp = dp.drop(columns=['Rank', 'G'])

stat_name_input = "Earned Run Average"
era = get_stat_dataframe(stat_name_input)
era.rename(columns={"R":"RA"}, inplace=True)
era = era.drop(columns=['Rank', 'G'])

stat_name_input = "Fielding Percentage"
fp = get_stat_dataframe(stat_name_input)
fp["APG"] = fp["A"] / fp["G"]
fp["EPG"] = fp["E"] / fp["G"]
fp = fp.drop(columns=['Rank', 'G'])

stat_name_input = "Hits Allowed Per Nine Innings"
ha = get_stat_dataframe(stat_name_input)
ha.rename(columns={"PG": "HAPG"}, inplace=True)
ha = ha.drop(columns=['Rank', 'G', 'IP'])

stat_name_input = "Home Runs Per Game"
hr = get_stat_dataframe(stat_name_input)
hr.rename(columns={"PG": "HRPG"}, inplace=True)
hr = hr.drop(columns=['Rank', 'G'])

stat_name_input = "On Base Percentage"
obp = get_stat_dataframe(stat_name_input)
obp.rename(columns={"PCT": "OBP"}, inplace=True)
obp["HBPPG"] = obp["HBP"] / obp["G"]
obp = obp.drop(columns=['Rank', 'G', 'AB', 'H', 'BB', 'SF', 'SH'])

stat_name_input = "Runs"
runs = get_stat_dataframe(stat_name_input)
runs["RPG"] = runs["R"] / runs["G"]
runs.rename(columns={"R": "RS"}, inplace=True)
runs = runs.drop(columns=['Rank', 'G'])

stat_name_input = "Sacrifice Bunts"
sb = get_stat_dataframe(stat_name_input)
sb.rename(columns={"SH": "SB"}, inplace=True)
sb["SBPG"] = sb["SB"] / sb["G"]
sb = sb.drop(columns=['Rank', 'G'])

stat_name_input = "Sacrifice Flies"
sf = get_stat_dataframe(stat_name_input)
sf["SFPG"] = sf["SF"] / sf["G"]
sf = sf.drop(columns=['Rank', 'G'])

stat_name_input = "Slugging Percentage"
slg = get_stat_dataframe(stat_name_input)
slg.rename(columns={"SLG PCT": "SLG"}, inplace=True)
slg = slg.drop(columns=['Rank', 'G', 'AB'])

stat_name_input = "Stolen Bases"
stl = get_stat_dataframe(stat_name_input)
stl["STLP"] = stl["SB"] / (stl["SB"] + stl["CS"])
stl["STLPG"] = stl["SB"] / stl["G"]
stl["CSPG"] = stl["CS"] / stl["G"]
stl["SAPG"] = (stl["SB"] + stl["CS"]) / stl["G"]
stl.rename(columns={"SB": "STL"}, inplace=True)
stl = stl.drop(columns=['Rank', 'G'])

stat_name_input = "Strikeout-to-Walk Ratio"
kbb = get_stat_dataframe(stat_name_input)
kbb["IP"] = round(kbb["IP"])
kbb.rename(columns={"K/BB": "KBB"}, inplace=True)
kbb.rename(columns={"BB": "PBB"}, inplace=True)
kbb = kbb.drop(columns=['Rank', 'App', 'IP'])

stat_name_input = "Strikeouts Per Nine Innings"
kp9 = get_stat_dataframe(stat_name_input)
kp9.rename(columns={"K/9": "KP9"}, inplace=True)
kp9 = kp9.drop(columns=['Rank', 'G', 'IP', 'SO'])

stat_name_input = "Walks Allowed Per Nine Innings"
wp9 = get_stat_dataframe(stat_name_input)
wp9.rename(columns={"PG": "WP9"}, inplace=True)
wp9 = wp9.drop(columns=['Rank', 'G', 'IP', 'BB'])

stat_name_input = "WHIP"
whip = get_stat_dataframe(stat_name_input)
whip = whip.drop(columns=['Rank', 'HA', 'IP', 'BB'])

dfs = [ba, bb, dp, era, fp, ha, hr, obp, runs, sb, sf, slg, stl, kbb, kp9, wp9, whip]
df_combined = pd.concat(dfs, axis=1, join="inner")
baseball_stats = df_combined.loc[:, ~df_combined.columns.duplicated()].sort_values('Team').reset_index(drop=True)

Batting Average Done
Base on Balls Done
Double Plays Per Game Done
Earned Run Average Done
Fielding Percentage Done
Hits Allowed Per Nine Innings Done
Home Runs Per Game Done
On Base Percentage Done
Runs Done
Sacrifice Bunts Done
Sacrifice Flies Done
Slugging Percentage Done
Stolen Bases Done
Strikeout-to-Walk Ratio Done
Strikeouts Per Nine Innings Done
Walks Allowed Per Nine Innings Done
WHIP Done


In [4]:
modeling_stats = baseball_stats[['Team', 'HPG', 'ABPG', 'HPAB', 
                'BBPG', 'ERA', 'PCT', 'APG', 
                'EPG', 'HAPG', 'HRPG', 'OBP', 
                'HBPPG', 'RPG', 'SBPG', 'SFPG', 
                'SLG', 'STLP', 'STLPG', 'CSPG', 
                'SAPG', 'KP9', 'WP9', 'WHIP']]
modeling_stats

,Team,HPG,ABPG,HPAB,BBPG,ERA,PCT,APG,EPG,HAPG,...,SBPG,SFPG,SLG,STLP,STLPG,CSPG,SAPG,KP9,WP9,WHIP
0,A&M-Corpus Christi,9.385965,33.631579,0.279082,4.233333,5.96,0.970,9.480769,1.096154,9.77,...,0.350000,0.421053,0.446,0.737500,1.072727,0.381818,1.454545,8.3,4.69,1.61
1,Abilene Christian,9.241379,32.758621,0.282105,4.534483,5.83,0.971,9.600000,1.036364,9.64,...,0.410714,0.400000,0.453,0.840000,1.125000,0.214286,1.339286,8.4,4.54,1.57
2,Air Force,9.654545,33.709091,0.286408,5.055556,5.64,0.972,9.031746,1.000000,9.45,...,0.472727,0.438596,0.467,0.771739,1.290909,0.381818,1.672727,8.6,4.42,1.54
3,Akron,8.314815,33.277778,0.249861,4.159091,8.35,0.955,10.314815,1.685185,11.93,...,0.120690,0.358974,0.376,0.764706,0.464286,0.142857,0.607143,6.7,6.53,2.00
4,Alabama,10.385965,34.403509,0.301887,6.078431,4.77,0.977,8.092593,0.796296,8.66,...,0.642857,0.542373,0.510,0.776860,1.540984,0.442623,1.983607,9.6,3.82,1.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,Wofford,11.548387,34.548387,0.334267,6.459016,3.79,0.982,10.035088,0.666667,7.31,...,1.214286,0.694915,0.607,0.798030,2.892857,0.732143,3.625000,11.9,2.91,1.23
291,Wright St.,10.982143,35.232143,0.311708,5.758621,4.35,0.979,9.490196,0.764706,8.11,...,0.851852,0.625000,0.538,0.751592,2.034483,0.672414,2.706897,10.2,3.42,1.33
292,Xavier,9.315789,34.315789,0.271472,4.370370,6.48,0.968,10.000000,1.169492,10.18,...,0.367347,0.385965,0.422,0.809524,0.894737,0.210526,1.105263,7.9,4.95,1.67
293,Yale,8.317073,32.390244,0.256777,3.923077,7.64,0.961,9.836364,1.509091,11.22,...,0.250000,0.354167,0.386,0.750000,0.734694,0.244898,0.979592,7.0,5.91,1.86


Data Dictionary

- G: Games
- AB: At Bats
- H: Hits
- BA: Batting Average
- HPG: Hits Per Game
- ABPG: At Bats Per Game
- HPAB: Hits Per At Bat
- BB: Walks
- BBPG: Walks Per Game
- DP: Double Plays
- DPPG: Double Plays Per Game
- IP: Innings Pitched
- RA: Runs Allowed
- ER: Earned Runs
- ERA: Earned Runs Allowed
- PO: Put Outs
- A: Assists
- E: Errors
- PCT: Fielding Percentage
- APG: Assists Per Game
- EPG: Errors Per Game
- HA: Hits Allowed
- HAPG: Hits Allowed Per Game
- HR: Home Runs Hit
- HRPG: Home Runs Hit Per Game
- HBP: Hit By Pitch
- OBP: On Base Percentage
- HBPPG: Hit By Pitch Per Game
- RS: Runs Scored
- RPG: Runs Scored Per Game
- SB: Sacrifice Bunts
- SBPG: Sacrifice Bunts Per Game
- SF: Sacrifice Flies
- SFPG: Sacrifice Flies Per Game
- TB: Total Bases
- SLG: Slugging Percentage
- STL: Stolen Bases
- CS: Caught Stealing
- STLP: Stolen Bases Success Percentage
- STLPG: Stolen Bases Per Game
- CSPG: Caught Stealing Per Game
- SAPG: Stealing Attempts Per Game
- SO: Pitching Strike Outs
- PBB: Pitching Walks
- KBB: Strikeouts to Walk Ratio
- KP9: Strikeouts Per Nine
- WP9: Walks Allowed Per Nine
- WHIP: Walks Hits Over Innings Pitched

In [ ]:
# # Scrape all stats at once
# for stat_name, url in stat_links.items():
#     print(f"Scraping: {stat_name} ({url})")
    
#     # Get stats page content
#     soup = get_soup(url)
    
#     # Locate table
#     table = soup.find("table")
#     if not table:
#         print(f"No table found for {stat_name}")
#         continue

#     # Extract table headers
#     headers = [th.text.strip() for th in table.find_all("th")]

#     # Extract table rows
#     data = []
#     for row in table.find_all("tr")[1:]:  # Skip header row
#         cols = row.find_all("td")
#         data.append([col.text.strip() for col in cols])

#     # Convert to DataFrame and save
#     df = pd.DataFrame(data, columns=headers)
#     # df.to_csv(f"{stat_name}.csv", index=False)
#     print(f"Saved {stat_name}.csv")

# print("Scraping completed!")